<a href="https://colab.research.google.com/github/JefersonCardona/Algoritmia-Programacion/blob/main/docuc%C3%B3digo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
from datetime import datetime
import logging

In [2]:
# Definimos la ruta final donde se guardará el archivo de log
RutaFinal = 'Ruta_final'

# Verificamos si la ruta final existe, si no, la creamos
if not os.path.exists(RutaFinal):
    os.makedirs(RutaFinal)

# Creamos la ruta completa del archivo de log
ArchivoLog = os.path.join(RutaFinal, 'asignaciones.log')

# Inicializamos una lista para almacenar los mensajes del log
MensajesLog = []

# Definimos una función para manejar los mensajes de log
def MensajeLog(L, Mensaje):
    # Añadimos el mensaje a la lista con un timestamp y el nivel de log
    MensajesLog.append(f"{datetime.now()} - {L} - {Mensaje}")

    # Imprimimos el mensaje en la consola si el nivel es INFO o ERROR
    if L == "INFO":
        print(f"{datetime.now()} - INFO - {Mensaje}")
    elif L == "ERROR":
        print(f"{datetime.now()} - ERROR - {Mensaje}")

# Registro inicial en el log
MensajeLog("INFO", "Cargando datos de Estudiantes y Malla Curricular")

# Carga de datos de estudiantes desde un archivo CSV remoto
# Se omite la columna 'Fecha' en los datos cargados
Datos = pd.read_csv('https://github.com/JefersonCardona/Algoritmia-Programacion/raw/eb776f421bb990b8c16ded477572f5d436f1f646/Archivos/Estudiantes.csv', encoding='latin1', delimiter=';').drop(columns=['Fecha'])

# Carga de la malla curricular desde un archivo CSV remoto
MallaCurricular = pd.read_csv('https://github.com/JefersonCardona/Algoritmia-Programacion/raw/eb776f421bb990b8c16ded477572f5d436f1f646/Archivos/MallaCurricular.csv',encoding='latin1',delimiter=';')

# Registro en el log después de cargar los datos
MensajeLog("INFO", "Datos cargados con éxito")

def AsignaturaCode(asignatura, semestre, creditos, consecutivo):
    """
    Crea un código para una asignatura con información básica.

    Parámetros:
    - asignatura: El nombre de la asignatura.
    - semestre: En qué semestre se da la asignatura.
    - creditos: Cuántos créditos vale la asignatura.
    - consecutivo: Un número para diferenciar asignaturas similares.

    Retorna:
    - Un código como texto que combina la información dada.
    """
    # Combina las primeras tres letras de la asignatura en mayúsculas,
    # el número del semestre, los créditos y un número consecutivo.
    return f"{asignatura[:3].upper()}{semestre}{creditos}{consecutivo:02d}"

def HTD(creditos):
    """
    Calcula las Horas de Trabajo del Docente (HTD) basadas en los créditos de una asignatura.

    Parámetros:
    - creditos: Número de créditos de la asignatura.

    Devuelve:
    - El número de horas de trabajo del docente correspondiente a los créditos.
    """
    # Se retorna un valor fijo de horas dependiendo del número de créditos.
    if creditos == 4:
        return 96
    elif creditos == 3:
        return 64
    elif creditos == 2:
        return 32
    elif creditos == 1:
        return 16
    # Si no coincide con ningún caso anterior, se retorna 0.
    return 0

def HTI(creditos):
    """
    Calcula las Horas de Trabajo Independiente (HTI) basadas en los créditos de una asignatura.

    Parámetros:
    - creditos: Número de créditos de la asignatura.

    Devuelve:
    - El número de horas de trabajo independiente correspondiente a los créditos.
    """
    # Se retorna un valor fijo de horas dependiendo del número de créditos.
    if creditos == 4:
        return 120
    elif creditos == 3:
        return 80
    elif creditos == 2:
        return 64
    elif creditos == 1:
        return 32
    # Si no coincide con ningún caso anterior, se retorna 0.
    return 0

    # Definimos la función principal que se encargará de matricular estudiantes.
def FuncionPrincipal(estudiantes_df, asignaturas_df, semestre, nivel, tamano_grupo):
    # Registramos un mensaje en el log para indicar el inicio del proceso de matriculación.
    MensajeLog("INFO", f"Matriculando estudiantes para el semestre {semestre}, nivel {nivel}")

    # Filtramos los estudiantes que corresponden al semestre actual.
    estudiantes_semestre = estudiantes_df[estudiantes_df['Semestre'] == semestre]

    # Contamos el total de estudiantes para este semestre.
    total_estudiantes_semestre = len(estudiantes_semestre)

    # Filtramos las asignaturas que corresponden al nivel actual.
    asignaturas_nivel = asignaturas_df[asignaturas_df['Nivel'] == nivel]

    # Creamos grupos de estudiantes con el tamaño especificado.
    grupos = [estudiantes_semestre[i:i + tamano_grupo] for i in range(0, len(estudiantes_semestre), tamano_grupo)]

    # Preparamos el directorio donde se guardarán las asignaciones.
    Directorio = os.path.join(RutaFinal, f'Asignaciones_Semestre_{semestre}')

    # Si el directorio no existe, lo creamos.
    if not os.path.exists(Directorio):
        os.makedirs(Directorio)
        MensajeLog("INFO", f"Directorio creado: {Directorio}")

    # Iteramos sobre cada asignatura del nivel actual.
    for index, asignatura in asignaturas_nivel.iterrows():
        # Obtenemos el nombre y los créditos de la asignatura.
        asignatura_nombre = asignatura['Asignatura']
        creditos = asignatura['Creditos']

        # Calculamos el total de cursos asignados.
        total_cursos_asignados = len(grupos)

        # Obtenemos la fecha actual en formato de año, mes y día.
        fecha_creacion = datetime.now().strftime('%Y%m%d')

        # Preparamos el directorio para la asignatura actual.
        asignatura_dir = os.path.join(Directorio, asignatura_nombre)

        # Si el directorio de la asignatura no existe, lo creamos.
        if not os.path.exists(asignatura_dir):
            os.makedirs(asignatura_dir)
            MensajeLog("INFO", f"Directorio creado: {asignatura_dir}")

        # Iteramos sobre cada grupo de estudiantes.
        for numero_grupo, grupo in enumerate(grupos, start=1):
            # Generamos el código de la asignatura.
            codigo_asignatura = AsignaturaCode(asignatura_nombre, semestre, creditos, numero_grupo)

            # Calculamos las horas de trabajo docente e independiente basadas en los créditos.
            horas_trabajo_docente = HTD(creditos)
            horas_trabajo_independiente = HTI(creditos)

            # Establecemos el número total de estudiantes y la cantidad de estudiantes en el grupo actual.
            numero_total_estudiantes = total_estudiantes_semestre
            cantidad_estudiantes = len(grupo)

            # Formateamos el nombre de la asignatura para el nombre del archivo.
            nombre_curso_formateado = asignatura_nombre.replace(" ", "").capitalize()

            # Preparamos los nombres de los archivos CSV y Excel para el grupo actual.
            grupo_filename = f"{codigo_asignatura}-{nombre_curso_formateado}-{cantidad_estudiantes}-{numero_grupo}.csv"
            grupo_excel_filename = f"{codigo_asignatura}-{nombre_curso_formateado}-{cantidad_estudiantes}-{numero_grupo}.xlsx"

            # Establecemos las rutas completas para los archivos del grupo.
            grupo_path = os.path.join(asignatura_dir, grupo_filename)
            grupo_excel_path = os.path.join(asignatura_dir, grupo_excel_filename)

            # Creamos un DataFrame con la información del grupo y lo guardamos en los archivos correspondientes.
            grupo_df = pd.DataFrame({'Estudiante': grupo['Nombre'], '(CA)': codigo_asignatura, '(HTD)': horas_trabajo_docente, '(HTI)': horas_trabajo_independiente, '(NTE)': numero_total_estudiantes, '(CC)': numero_grupo, '(TCA)': total_cursos_asignados, '(FC)': fecha_creacion })
            grupo_df.to_csv(grupo_path, index=False, encoding='latin1')
            grupo_df.to_excel(grupo_excel_path, index=False)

            # Registramos un mensaje en el log para cada grupo generado.
            MensajeLog("INFO", f"Archivos generados para la asignatura {asignatura_nombre}, grupo {numero_grupo}")

    # Registramos un mensaje final en el log para indicar que se han generado todos los archivos CSV.
    MensajeLog("INFO", f"Archivos CSV de asignaciones generados para el semestre {semestre}")


2024-06-07 03:42:42.538705 - INFO - Cargando datos de Estudiantes y Malla Curricular
2024-06-07 03:42:43.290764 - INFO - Datos cargados con éxito


In [3]:
# Aquí estamos llamando a la función 'FuncionPrincipal' varias veces con diferentes parámetros.
# Cada llamada corresponde a un semestre y nivel específico, con un tamaño de grupo determinado.

# Matriculamos estudiantes de primer semestre de nivel 1 en grupos de 30.
FuncionPrincipal(Datos, MallaCurricular, semestre=1, nivel=1, tamano_grupo=30)

# Matriculamos estudiantes de segundo semestre de nivel 2 en grupos de 30.
FuncionPrincipal(Datos, MallaCurricular, semestre=2, nivel=2, tamano_grupo=30)

# Continuamos este proceso hasta el décimo semestre, ajustando el tamaño del grupo según sea necesario.
FuncionPrincipal(Datos, MallaCurricular, semestre=3, nivel=3, tamano_grupo=30)
FuncionPrincipal(Datos, MallaCurricular, semestre=4, nivel=4, tamano_grupo=25)
FuncionPrincipal(Datos, MallaCurricular, semestre=5, nivel=5, tamano_grupo=25)
FuncionPrincipal(Datos, MallaCurricular, semestre=6, nivel=6, tamano_grupo=25)
FuncionPrincipal(Datos, MallaCurricular, semestre=7, nivel=7, tamano_grupo=20)
FuncionPrincipal(Datos, MallaCurricular, semestre=8, nivel=8, tamano_grupo=20)
FuncionPrincipal(Datos, MallaCurricular, semestre=9, nivel=9, tamano_grupo=20)
FuncionPrincipal(Datos, MallaCurricular, semestre=10, nivel=10, tamano_grupo=10)

# Después de matricular a los estudiantes, procedemos a escribir los mensajes de registro.
# Abrimos el archivo de registro en modo de escritura.
with open(ArchivoLog, 'w') as log_file:
    # Iteramos sobre cada mensaje en la lista de mensajes de registro.
    for Mensaje in MensajesLog:
        # Escribimos cada mensaje en el archivo de registro seguido de un salto de línea.
        log_file.write(Mensaje + '\n')


2024-06-07 03:42:51.549310 - INFO - Matriculando estudiantes para el semestre 1, nivel 1
2024-06-07 03:42:51.552598 - INFO - Directorio creado: Ruta_final/Asignaciones_Semestre_1
2024-06-07 03:42:51.553218 - INFO - Directorio creado: Ruta_final/Asignaciones_Semestre_1/Algebra y Trigonometria
2024-06-07 03:42:51.871994 - INFO - Archivos generados para la asignatura Algebra y Trigonometria, grupo 1
2024-06-07 03:42:51.889620 - INFO - Archivos generados para la asignatura Algebra y Trigonometria, grupo 2
2024-06-07 03:42:51.906464 - INFO - Archivos generados para la asignatura Algebra y Trigonometria, grupo 3
2024-06-07 03:42:51.925229 - INFO - Archivos generados para la asignatura Algebra y Trigonometria, grupo 4
2024-06-07 03:42:51.940744 - INFO - Archivos generados para la asignatura Algebra y Trigonometria, grupo 5
2024-06-07 03:42:51.941905 - INFO - Directorio creado: Ruta_final/Asignaciones_Semestre_1/Calculo Diferencial
2024-06-07 03:42:51.957522 - INFO - Archivos generados para la